In [1]:
# This creates odds and ownership df for projections. Maybe seaparate

In [2]:
import os
import numpy as np
import pandas as pd
import requests
from datetime import date
baseball_path = r"C:\Users\james\Documents\MLB\Data"

In [3]:
todaysdate = date.today()
todaysdate_dash = str(todaysdate)
todaysdate = todaysdate_dash.replace("-", "")

# todaysdate = "20230330"
print(todaysdate)

20230428


In [4]:
# This reads in a map of team name, codes, and the number Fangraphs uses in their URLs
team_map = pd.read_csv(os.path.join(baseball_path, "Utilities", "Team Map.csv"))

# We just need teams right now
team_map = team_map[['FULLNAME', 'FANPROSTEAM', 'BBREFTEAM']]

In [5]:
# Has a lot more fun data too! Including, weirdly, team colors! Might want to add more to this later
def gamble_df(jsonData):
    list_list = []
    for i in range(len(jsonData)):
        home_name = jsonData[i]['EventDetails']['Properties']['HomeTeamShort']
        away_name = jsonData[i]['EventDetails']['Properties']['VisitorTeamShort']
        home_runs = jsonData[i]['EventDetails']['Properties']['HomeVegasRuns']
        away_runs = jsonData[i]['EventDetails']['Properties']['VisitorVegasRuns']

        away_ml = jsonData[i]['EventDetails']['Properties']['VisitorGameMoneylineCurrent']
        home_ml = jsonData[i]['EventDetails']['Properties']['HomeGameMoneylineCurrent']
        
        home_ou = jsonData[i]['EventDetails']['Properties']['HomeGameOUCurrent']
        home_ou_juice = jsonData[i]['EventDetails']['Properties']['HomeGameOUJuiceCurrent']

        home_spread = jsonData[i]['EventDetails']['Properties']['HomeGameSpreadCurrent']
        home_spread_juice = jsonData[i]['EventDetails']['Properties']['HomeGameSpreadJuiceCurrent']

        date = jsonData[i]['EventDetails']['Properties']['EventDateId']

        gamble_list = [away_name, away_runs, home_name, home_runs, home_ou, home_ou_juice, home_spread, home_spread_juice, away_ml, home_ml, date]
        list_list.append(gamble_list)
    
    df = pd.DataFrame(list_list, columns=['away_team','away_runs', 'home_team', 'home_runs', 'ou', 'ou_juice', 'spread', 'spread_juice', 'away_ml', 'home_ml', 'date'])
    
    return df

In [6]:
def daily_gamble(date):
    print(date)
    dateStr = date[0:4] + "-" + date[4:6] + "-" + date[6:8]
    
    url = f'https://www.fantasylabs.com/api/sportevents/3/{dateStr}/vegas/?sportId=3&date={dateStr}'
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36'}
    jsonData = requests.get(url, headers=headers).json()
    
    # try:
    df = gamble_df(jsonData)
    
    
    df = df.merge(team_map, left_on='away_team', right_on='FANPROSTEAM', how='left')
    df['away_team'] = df['BBREFTEAM']
    df.drop(columns={'FULLNAME', 'FANPROSTEAM', 'BBREFTEAM'}, inplace=True)
    
    df = df.merge(team_map, left_on='home_team', right_on='FANPROSTEAM', how='left')
    df['home_team'] = df['BBREFTEAM']
    df.drop(columns={'FULLNAME', 'FANPROSTEAM', 'BBREFTEAM'}, inplace=True)

    odds_filename = "Odds " + date + ".csv"
    df.to_csv(os.path.join(baseball_path, "Odds", odds_filename))
    
daily_gamble(todaysdate)

20230428


In [7]:
# # Create all odds DFs
# import time
# for filename in os.listdir(r"C:\Users\james\Documents\MLB\Data\A1. Depth Charts"):
#     date = filename[5:13]
#     print(date)
    
#     dateStr = date[0:4] + "-" + date[4:6] + "-" + date[6:8]
    
#     url = f'https://www.fantasylabs.com/api/sportevents/3/{dateStr}/vegas/?sportId=3&date={dateStr}'
#     headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36'}
#     jsonData = requests.get(url, headers=headers).json()
    
#     try:
#         df = daily_gamble(date)
#         time.sleep(6)
#     except:
#         pass

In [8]:
# # Create all odds DFs
# import time
# for filename in os.listdir(r"C:\Users\james\Documents\MLB\Data\A1. Depth Charts"):
#     date = filename[5:13]
#     print(date)
    
#     dateStr = date[0:4] + "-" + date[4:6] + "-" + date[6:8]
    
#     url = f'https://www.fantasylabs.com/api/sportevents/3/{dateStr}/vegas/?sportId=3&date={dateStr}'
#     headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36'}
#     jsonData = requests.get(url, headers=headers).json()
    
#     try:
#         df = gamble_df(jsonData)
#         odds_filename = "Odds " + date + ".csv"
#         df.to_csv(os.path.join(baseball_path, "Odds", odds_filename))
#         time.sleep(6)
        
#     except:
#         pass